In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetimes

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [4]:
NSAIDs = pd.read_excel('data\\NSAIDs_ATCcode.xlsx')
NSAIDs

,code
0,M01AA Butylpyrazolidines
1,M01AA01 Pyrazinobutazone
2,M01AB01 Indometacin
3,M01AB02 Sulindac
4,M01AB03 Tolmetin
...,...
79,N02CC02 Naratriptan
80,N02CC03 Zolmitriptan
81,N02CC05 Almotriptan
82,N03AX Other antiepileptics


In [5]:
NSAIDs[['ATC_code', 'dump']] = NSAIDs['code'].str.split(' ', 1, expand=True)
NSAIDs

,code,ATC_code,dump
0,M01AA Butylpyrazolidines,M01AA,Butylpyrazolidines
1,M01AA01 Pyrazinobutazone,M01AA01,Pyrazinobutazone
2,M01AB01 Indometacin,M01AB01,Indometacin
3,M01AB02 Sulindac,M01AB02,Sulindac
4,M01AB03 Tolmetin,M01AB03,Tolmetin
...,...,...,...
79,N02CC02 Naratriptan,N02CC02,Naratriptan
80,N02CC03 Zolmitriptan,N02CC03,Zolmitriptan
81,N02CC05 Almotriptan,N02CC05,Almotriptan
82,N03AX Other antiepileptics,N03AX,Other antiepileptics


In [6]:
NSAIDs = NSAIDs[['ATC_code']]
NSAIDs

,ATC_code
0,M01AA
1,M01AA01
2,M01AB01
3,M01AB02
4,M01AB03
...,...
79,N02CC02
80,N02CC03
81,N02CC05
82,N03AX


In [8]:
NSAIDs.to_csv('data//NSAIDs_ATCcode_preprocessed.csv', encoding = 'euc-kr', index = False)

In [11]:
main_drugcode = pd.read_excel('data//약제급여품목 ATC코드 매핑 현황(2017.12.1.급여목록표 기준).xlsx')
main_drugcode

,약제급여품목의 ATC코드 매핑 현황(2017.12.1. 급여목록표기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,연번,분류,주성분코드,제품코드,제품명,업체명,ATC코드,ATC코드 명칭,비고
2,1,112,130830ASY,645302132,포크랄시럽(포수클로랄)_(9.5g/95mL),한림제약(주),N05CC01,chloral hydrate,NaN
3,2,112,130831ASY,645302131,포크랄시럽(포수클로랄)_(10g/100mL),한림제약(주),N05CC01,chloral hydrate,NaN
4,3,112,149203ATB,677100550,고나스정3mg(독세핀염산염)_(3.39mg/1정),(주)펜믹스,N06AA12,doxepin,NaN
...,...,...,...,...,...,...,...,...,...
22295,22294,821,630901CSI,641605801,인스타닐나잘스프레이50마이크로그램(펜타닐시트르산염)_(0.5mg/10회),(주)대웅제약,N02AB03,fentanyl,NaN
22296,22295,332,238201DGN,658601560,이연트롬빈동결건조분말5000단위_(5KI.U/1병),이연제약(주),B02BC06,thrombin,NaN
22297,22296,341,393301DGN,646601660,크린카트에이(무수탄산나트륨)_(13g/1관),(주)박스터,V07AV,Technical disinfectants,NaN
22298,22297,341,408401DGN,646601680,크린카트씨(무수구연산)_(32g/1관),(주)박스터,V07AV,Technical disinfectants,NaN


In [12]:
# 열 이름을 두번째 행으로 지정하기 

main_drugcode.columns = main_drugcode.iloc[1]
main_drugcode

1,연번,분류,주성분코드,제품코드,제품명,업체명,ATC코드,ATC코드 명칭,비고
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,연번,분류,주성분코드,제품코드,제품명,업체명,ATC코드,ATC코드 명칭,비고
2,1,112,130830ASY,645302132,포크랄시럽(포수클로랄)_(9.5g/95mL),한림제약(주),N05CC01,chloral hydrate,NaN
3,2,112,130831ASY,645302131,포크랄시럽(포수클로랄)_(10g/100mL),한림제약(주),N05CC01,chloral hydrate,NaN
4,3,112,149203ATB,677100550,고나스정3mg(독세핀염산염)_(3.39mg/1정),(주)펜믹스,N06AA12,doxepin,NaN
...,...,...,...,...,...,...,...,...,...
22295,22294,821,630901CSI,641605801,인스타닐나잘스프레이50마이크로그램(펜타닐시트르산염)_(0.5mg/10회),(주)대웅제약,N02AB03,fentanyl,NaN
22296,22295,332,238201DGN,658601560,이연트롬빈동결건조분말5000단위_(5KI.U/1병),이연제약(주),B02BC06,thrombin,NaN
22297,22296,341,393301DGN,646601660,크린카트에이(무수탄산나트륨)_(13g/1관),(주)박스터,V07AV,Technical disinfectants,NaN
22298,22297,341,408401DGN,646601680,크린카트씨(무수구연산)_(32g/1관),(주)박스터,V07AV,Technical disinfectants,NaN


In [13]:
main_drugcode.drop([0, 1], axis = 0, inplace = True)
main_drugcode.reset_index(drop = True)

1,연번,분류,주성분코드,제품코드,제품명,업체명,ATC코드,ATC코드 명칭,비고
0,1,112,130830ASY,645302132,포크랄시럽(포수클로랄)_(9.5g/95mL),한림제약(주),N05CC01,chloral hydrate,NaN
1,2,112,130831ASY,645302131,포크랄시럽(포수클로랄)_(10g/100mL),한림제약(주),N05CC01,chloral hydrate,NaN
2,3,112,149203ATB,677100550,고나스정3mg(독세핀염산염)_(3.39mg/1정),(주)펜믹스,N06AA12,doxepin,NaN
3,4,112,149203ATB,651904420,명세핀정3밀리그램(독세핀염산염)_(3.39mg/1정),명인제약(주),N06AA12,doxepin,NaN
4,5,112,149203ATB,649001620,리치나잇정3밀리그램(독세핀염산염)_(3.39mg/1정),풍림무약(주),N06AA12,doxepin,NaN
...,...,...,...,...,...,...,...,...,...
22293,22294,821,630901CSI,641605801,인스타닐나잘스프레이50마이크로그램(펜타닐시트르산염)_(0.5mg/10회),(주)대웅제약,N02AB03,fentanyl,NaN
22294,22295,332,238201DGN,658601560,이연트롬빈동결건조분말5000단위_(5KI.U/1병),이연제약(주),B02BC06,thrombin,NaN
22295,22296,341,393301DGN,646601660,크린카트에이(무수탄산나트륨)_(13g/1관),(주)박스터,V07AV,Technical disinfectants,NaN
22296,22297,341,408401DGN,646601680,크린카트씨(무수구연산)_(32g/1관),(주)박스터,V07AV,Technical disinfectants,NaN


In [19]:
main_drugcode.rename(columns = {'ATC코드' : 'ATC_code'}, inplace = True)
main_drugcode

1,연번,분류,주성분코드,제품코드,제품명,업체명,ATC_code,ATC코드 명칭,비고
2,1,112,130830ASY,645302132,포크랄시럽(포수클로랄)_(9.5g/95mL),한림제약(주),N05CC01,chloral hydrate,NaN
3,2,112,130831ASY,645302131,포크랄시럽(포수클로랄)_(10g/100mL),한림제약(주),N05CC01,chloral hydrate,NaN
4,3,112,149203ATB,677100550,고나스정3mg(독세핀염산염)_(3.39mg/1정),(주)펜믹스,N06AA12,doxepin,NaN
5,4,112,149203ATB,651904420,명세핀정3밀리그램(독세핀염산염)_(3.39mg/1정),명인제약(주),N06AA12,doxepin,NaN
6,5,112,149203ATB,649001620,리치나잇정3밀리그램(독세핀염산염)_(3.39mg/1정),풍림무약(주),N06AA12,doxepin,NaN
...,...,...,...,...,...,...,...,...,...
22295,22294,821,630901CSI,641605801,인스타닐나잘스프레이50마이크로그램(펜타닐시트르산염)_(0.5mg/10회),(주)대웅제약,N02AB03,fentanyl,NaN
22296,22295,332,238201DGN,658601560,이연트롬빈동결건조분말5000단위_(5KI.U/1병),이연제약(주),B02BC06,thrombin,NaN
22297,22296,341,393301DGN,646601660,크린카트에이(무수탄산나트륨)_(13g/1관),(주)박스터,V07AV,Technical disinfectants,NaN
22298,22297,341,408401DGN,646601680,크린카트씨(무수구연산)_(32g/1관),(주)박스터,V07AV,Technical disinfectants,NaN


In [21]:
final = pd.merge(NSAIDs, main_drugcode, how = 'inner', on = 'ATC_code')

In [23]:
final.head(30)

,ATC_code,연번,분류,주성분코드,제품코드,제품명,업체명,ATC코드 명칭,비고
0,M01AB02,1029,114,233201ATB,646801230,크리돌정100밀리그람(설린닥)_(0.1g/1정),건일제약(주),sulindac,NaN
1,M01AB02,1030,114,233203ATB,646801240,크리돌정200mg(설린닥)_(0.2g/1정),건일제약(주),sulindac,NaN
2,M01AB05,653,114,143402ATB,657801710,카타스정50밀리그람(디클로페낙칼륨)_(50mg/1정),하나제약(주),diclofenac,NaN
3,M01AB05,654,114,143501ATR,657801720,카페낙서방정100㎎(디클로페낙나트륨)(수출명:볼맥스서방정)_(0.1g/1정),하나제약(주),diclofenac,NaN
4,M01AB05,655,114,143502ATB,665506280,모티펜캡슐75밀리그램(디클로페낙나트륨)_(75mg/1캡슐),한국유니온제약(주),diclofenac,NaN
5,M01AB05,656,114,143504ATB,649700140,디낙스정(디클로페낙나트륨)_(25mg/1정),크라운제약(주),diclofenac,NaN
6,M01AB05,657,114,143504ATB,645901100,디페인정(디클로페낙나트륨)_(25mg/1정),동광제약(주),diclofenac,NaN
7,M01AB05,658,114,143504ATB,653800990,신일디클로페낙나트륨정_(25mg/1정),신일제약(주),diclofenac,NaN
8,M01AB05,659,114,143506ATB,653801000,신일디클로페낙나트륨정50밀리그람_(50mg/1정),신일제약(주),diclofenac,NaN
9,M01AB05,15930,114,143534BIJ,648501881,신풍디클로페낙나트륨주(수출명: SHINPOONG CLOFEN Injection))_...,신풍제약(주),diclofenac,NaN


In [28]:
final_main = final[['주성분코드', 'ATC코드 명칭']].copy()
final_main['주성분'] = final_main['주성분코드'].str.slice(0,4)
final_main

,주성분코드,ATC코드 명칭,주성분
0,233201ATB,sulindac,2332
1,233203ATB,sulindac,2332
2,143402ATB,diclofenac,1434
3,143501ATR,diclofenac,1435
4,143502ATB,diclofenac,1435
...,...,...,...
967,233802ATB,sumatriptan,2338
968,233803ATB,sumatriptan,2338
969,415501ATB,naratriptan,4155
970,415601ATB,zolmitriptan,4156


In [29]:
final_main.drop_duplicates(subset='주성분', inplace = True)
final_main.reset_index(drop = True, inplace = True)
final_main

,주성분코드,ATC코드 명칭,주성분
0,233201ATB,sulindac,2332
1,143402ATB,diclofenac,1434
2,143501ATR,diclofenac,1435
3,143630BIJ,diclofenac,1436
4,156601ATB,etodolac,1566
5,101001ATB,acemetacin,1010
6,218602ATB,proglumetacin,2186
7,180001ATB,ketorolac,1800
8,100901ATB,aceclofenac,1009
9,133801ATB,Oxicams,1338


In [30]:
final_main.to_csv('data//NSAIDs_MainIngredientCode_preprocessed.csv', encoding = 'euc-kr', index = False)